In [4]:
import sympy as sp

# Global symbols
t = sp.symbols('t', real=True)
x = sp.Function('x')(t)
u = sp.Function('u')(t)
lam = sp.Function('lam')(t)
C1, C2 = sp.symbols('C1 C2')  # For general constants

In [5]:
def define_hamiltonian(lagrangian_expr, dynamics_expr):
    return lagrangian_expr + lam * dynamics_expr

def solve_costate(H, t_f=None, lam_tf=None, general=False):
    dHdx = sp.diff(H, x)
    ode = sp.Eq(sp.diff(lam, t), -dHdx)
    if general:
        sol = sp.dsolve(ode, lam)
    else:
        sol = sp.dsolve(ode, lam, ics={lam.subs(t, t_f): lam_tf})
    return sol.rhs

def solve_optimal_control(H, lam_expr):
    dHdu = sp.diff(H, u)
    u_sol = sp.solve(sp.Eq(dHdu, 0), u)[0]
    return u_sol.subs(lam, lam_expr)

def solve_state(f_dyn, u_expr, x0=None, t0=None, general=False):
    f_subbed = f_dyn.subs(u, u_expr)
    ode = sp.Eq(sp.diff(x, t), f_subbed)
    if general:
        sol = sp.dsolve(ode, x)
    else:
        sol = sp.dsolve(ode, x, ics={x.subs(t, t0): x0})
    return sol.rhs

def enforce_boundary_conditions(x_expr, t_0, x_0, t_f, x_f):
    C = list(x_expr.free_symbols & {C1, C2})
    eqs = [
        sp.Eq(x_expr.subs(t, t_0), x_0),
        sp.Eq(x_expr.subs(t, t_f), x_f)
    ]
    sol = sp.solve(eqs, C, dict=True)[0]
    return x_expr.subs(sol)

# ---------------------------- Main Entry Function ----------------------------

def pontryagin_pipeline(L, f_dyn, x_cond, t_int, lam_tf=None):
    """
    Unified Pontryagin pipeline:
    - If x_cond = (x0, xf): uses two-point boundary value solution
    - If x_cond = scalar: uses x0 and lam_tf for single-boundary case
    """
    t_0, t_f = t_int

    # Single or two-point condition?
    is_two_point = isinstance(x_cond, (tuple, list)) and len(x_cond) == 2

    if is_two_point:
        x_0, x_f = x_cond
        H = sp.simplify(define_hamiltonian(L, f_dyn))
        lam_expr = sp.simplify(solve_costate(H, general=True))
        u_expr = sp.simplify(solve_optimal_control(H, lam_expr))
        x_expr_gen = sp.simplify(solve_state(f_dyn, u_expr, general=True))
        x_expr_final = sp.simplify(enforce_boundary_conditions(x_expr_gen, t_0, x_0, t_f, x_f))
        return {
            'Hamiltonian': H,
            'lambda(t)': lam_expr,
            'u(t)': u_expr,
            'x_general(t)': x_expr_gen,
            'x(t)': x_expr_final
        }

    else:
        x_0 = x_cond
        H = sp.simplify(define_hamiltonian(L, f_dyn))
        lam_expr = sp.simplify(solve_costate(H, t_f, lam_tf, general=False))
        u_expr = sp.simplify(solve_optimal_control(H, lam_expr))
        x_expr = sp.simplify(solve_state(f_dyn, u_expr, x_0, t_0, general=False))
        return {
            'Hamiltonian': H,
            'lambda(t)': lam_expr,
            'u(t)': u_expr,
            'x(t)': x_expr
        }

In [6]:
problems = [
    {
        "label": "Problem 1",
        "L_expr": sp.exp(3*t)*x - u**2,
        "Dyn_expr": u,
        "x_cond": 3,
        "t_int": (0, 2),
        "lam_tf": 0
    },
    {
        "label": "Problem 2",
        "L_expr": x - u**2,
        "Dyn_expr": u - x,
        "x_cond": (2, sp.exp(2)),
        "t_int": (0, 3),
        "lam_tf": None
    },
    {
        "label": "Problem 3",
        "L_expr": 7*t**2 * x - u**2,
        "Dyn_expr": u,
        "x_cond": (0, 2),
        "t_int": (0, 2),
        "lam_tf": None
    },
]

for prob in problems:
    print("="*40)
    print(prob["label"])
    result = pontryagin_pipeline(
        prob["L_expr"], prob["Dyn_expr"], prob["x_cond"], prob["t_int"], prob["lam_tf"]
    )
    for key, val in result.items():
        print(f"{key}: {val}")
    print(sp.N(result['x(t)']))


Problem 1
Hamiltonian: lam(t)*u(t) - u(t)**2 + x(t)*exp(3*t)
lambda(t): -exp(3*t)/3 + exp(6)/3
u(t): -exp(3*t)/6 + exp(6)/6
x(t): t*exp(6)/6 - exp(3*t)/18 + 55/18
67.2381322487892*t - 0.0555555555555556*exp(3*t) + 3.05555555555556
Problem 2
Hamiltonian: (u(t) - x(t))*lam(t) - u(t)**2 + x(t)
lambda(t): C1*exp(t) + 1
u(t): C1*exp(t)/2 + 1/2
x_general(t): C1*exp(t)/4 + C2*exp(-t) + 1/2
x(t): ((-2*exp(5) + 3 + exp(3))*exp(2*t) + (1 - exp(6))*exp(t) + (-3*exp(3) - 1 + 2*exp(2))*exp(3))*exp(-t)/(2*(1 - exp(6)))
-0.00124245582842229*(-273.740781281966*exp(2*t) - 402.428793492735*exp(t) - 933.54559919624)*exp(-t)
Problem 3
Hamiltonian: 7*t**2*x(t) + lam(t)*u(t) - u(t)**2
lambda(t): C1 - 7*t**3/3
u(t): C1/2 - 7*t**3/6
x_general(t): C1*t/2 + C2 - 7*t**4/24
x(t): t*(80 - 7*t**3)/24
0.0416666666666667*t*(80.0 - 7.0*t**3)
